In [13]:
import pandas as pd
from sqlalchemy import create_engine
from config import username, password
import json
import requests 
from pprint import pprint
import time
import pandas as pd
from sqlalchemy import create_engine 
from flask import jsonify

In [14]:
rds_connection_string = f"{username}:{password}@localhost:5433/park_db"
engine = create_engine(f"postgresql://{rds_connection_string}")

In [15]:
engine.table_names()

['park',
 'webcam_url',
 'images',
 'fire',
 'sci_name',
 'park_species',
 'activity',
 'park_activities',
 'park_fee',
 'fees']

In [16]:
df = pd.read_sql_query("SELECT * FROM park_species WHERE category_name = 'Mammal' AND park_id = '6DA17C86-088E-4B4D-B862-7C1BD5CF236B'", con=engine)
df.head()

,sci_name_id,park_id,category_name,family,order,common_names
0,0,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,Moose
1,1,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,2,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Coyote, Eastern Coyote"
3,3,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."
4,4,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Mustelidae,Carnivora,"Otter, River Otter"


In [17]:
df_1 = pd.read_sql_query("SELECT p.park_name, p.park_id, \
ps.category_name, ps.family, ps.order, ps.common_names \
FROM park_species AS ps \
INNER JOIN park as p \
ON ps.park_id = p.park_id", con=engine)
df_1.head()

,park_name,park_id,category_name,family,order,common_names
0,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,Moose
1,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,"Northern White-Tailed Deer, Virginia Deer, Whi..."
2,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Coyote, Eastern Coyote"
3,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re..."
4,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Mustelidae,Carnivora,"Otter, River Otter"


In [18]:
species_df = pd.read_sql_query("""SELECT p.park_name, p.park_id, 
    ps.category_name, ps.family, ps.order, ps.common_names, s.sci_name 
    FROM park_species AS ps
    INNER JOIN park as p
    ON ps.park_id = p.park_id
    LEFT OUTER JOIN sci_name as s
    ON s.sci_name_id = ps.sci_name_id;""", con=engine)
species_df.head()

,park_name,park_id,category_name,family,order,common_names,sci_name
0,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,Moose,Alces alces
1,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Cervidae,Artiodactyla,"Northern White-Tailed Deer, Virginia Deer, Whi...",Odocoileus virginianus
2,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Coyote, Eastern Coyote",Canis latrans
3,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Canidae,Carnivora,"Black Fox, Cross Fox, Eastern Red Fox, Fox, Re...",Vulpes vulpes
4,Acadia National Park,6DA17C86-088E-4B4D-B862-7C1BD5CF236B,Mammal,Mustelidae,Carnivora,"Otter, River Otter",Lutra canadensis


In [20]:
new_json = (species_df.groupby(["park_name", "category_name"], as_index=True)
                .apply(lambda x: x[["order", "family", "sci_name", "common_names"]].to_dict("r")).reset_index()
                .rename(columns={0: "animals"})
                .to_json("test.json", orient="records"))